In [6]:
# Importando bibliotecas e dataset
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# Autenticação
api = KaggleApi()
api.authenticate()

dataset = 'fahmidachowdhury/manufacturing-defects'
file_name = 'defects_data.csv'
destination = './data'

In [7]:
# Cria pasta de destino se não existir
os.makedirs(destination, exist_ok=True)

# Baixa o arquivo CSV diretamente
api.dataset_download_file(dataset, file_name, path=destination, force=False)

print(f"{file_name} baixado com sucesso em {destination}!")

# Caminho do arquivo original
arquivo_original = './data/defects_data.csv'

Dataset URL: https://www.kaggle.com/datasets/fahmidachowdhury/manufacturing-defects
defects_data.csv baixado com sucesso em ./data!


In [8]:
# Lê o CSV e traduzir colunas
df = pd.read_csv(arquivo_original)

# 1) Converter a coluna de data para datetime (robusto para M/D/YYYY ou YYYY-MM-DD)
# Tenta primeiro formato americano, depois ISO; se já estiver em datetime, segue
try:
    df['defect_date'] = pd.to_datetime(df['defect_date'], format='%m/%d/%Y', errors='raise')
except Exception:
    df['defect_date'] = pd.to_datetime(df['defect_date'], errors='coerce')

# Verifica se houve alguma falha na conversão
if df['defect_date'].isna().any():
    linhas_invalidas = df[df['defect_date'].isna()]
    print("Aviso: algumas datas não puderam ser convertidas. Linhas afetadas:")
    print(linhas_invalidas.head())

# 2) Formatar para padrão brasileiro (substitui a coluna original)
df['defect_date'] = df['defect_date'].dt.strftime('%d/%m/%Y')

# 3) Remover coluna auxiliar se existir
if 'defect_date_br' in df.columns:
    df = df.drop(columns=['defect_date_br'])

# 4) Traduzir títulos das colunas
traducao_colunas = {
    'defect_id': 'id_defeito',
    'product_id': 'id_produto',
    'defect_type': 'tipo_defeito',
    'defect_date': 'data_defeito',
    'defect_location': 'local_defeito',
    'severity': 'severidade',
    'inspection_method': 'metodo_inspecao',
    'repair_cost': 'custo_reparo',
}
df = df.rename(columns=traducao_colunas)

# 5) Salvar no mesmo local com novo nome
pasta = os.path.dirname(arquivo_original)
novo_arquivo = os.path.join(pasta, 'defeitos_dados.csv')
df.to_csv(novo_arquivo, index=False, encoding='utf-8-sig')

print(f"Arquivo traduzido e com datas no padrão brasileiro salvo em: {novo_arquivo}")

Arquivo traduzido e com datas no padrão brasileiro salvo em: ./data\defeitos_dados.csv


In [9]:
df = pd.read_csv('./data\defeitos_dados.csv')

In [10]:
df.head()

,id_defeito,id_produto,tipo_defeito,data_defeito,local_defeito,severidade,metodo_inspecao,custo_reparo
0,1,15,Structural,06/06/2024,Component,Minor,Visual Inspection,245.47
1,2,6,Functional,26/04/2024,Component,Minor,Visual Inspection,26.87
2,3,84,Structural,15/02/2024,Internal,Minor,Automated Testing,835.81
3,4,10,Functional,28/03/2024,Internal,Critical,Automated Testing,444.47
4,5,14,Cosmetic,26/04/2024,Component,Minor,Manual Testing,823.64
